In [52]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1252')

In [54]:
#select relevant features
df1 = df[['iid', 'id', 'wave', 'gender', 'age','race', 'dec_o', 'imprace', 'imprelig', 'from', 'goal', 'date', 'go_out', 'career_c', 'sports',
'tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga', 'exphappy', 'attr3_1','sinc3_1',
'fun3_1','intel3_1', 'amb3_1', 'match_es']]
df1.head()

,iid,id,wave,gender,age,race,dec_o,imprace,imprelig,from,...,music,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es
0,1,1.0,1,0,21.0,4.0,0,2.0,4.0,Chicago,...,9.0,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0
1,1,1.0,1,0,21.0,4.0,0,2.0,4.0,Chicago,...,9.0,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0
2,1,1.0,1,0,21.0,4.0,1,2.0,4.0,Chicago,...,9.0,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0
3,1,1.0,1,0,21.0,4.0,1,2.0,4.0,Chicago,...,9.0,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0
4,1,1.0,1,0,21.0,4.0,1,2.0,4.0,Chicago,...,9.0,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 38 columns):
iid         8378 non-null int64
id          8377 non-null float64
wave        8378 non-null int64
gender      8378 non-null int64
age         8283 non-null float64
race        8315 non-null float64
dec_o       8378 non-null int64
imprace     8299 non-null float64
imprelig    8299 non-null float64
from        8299 non-null object
goal        8299 non-null float64
date        8281 non-null float64
go_out      8299 non-null float64
career_c    8240 non-null float64
sports      8299 non-null float64
tvsports    8299 non-null float64
exercise    8299 non-null float64
dining      8299 non-null float64
museums     8299 non-null float64
art         8299 non-null float64
hiking      8299 non-null float64
gaming      8299 non-null float64
clubbing    8299 non-null float64
reading     8299 non-null float64
tv          8299 non-null float64
theater     8299 non-null float64
movies      8299 n

In [55]:
#filling in NaNs
df1['id'].fillna(method ='ffill', inplace = True)
df1['from'].fillna("USA", inplace = True)

## create new df with aggregated columns

In [56]:
#create aggregated df and keeping the column names as they are 
df2 = df1.groupby(['iid', 'id', 'wave', 'gender', 'from'], as_index=False).agg({'age':['mean'], 'dec_o':['sum'], 'race':['mean'], 'imprace':['mean'], 'imprelig':['mean'], 'goal':['mean'], 'date':['mean'], 'go_out':['mean'], 'career_c':['mean'], 'sports':['mean'], 'tvsports':['mean'],'exercise':['mean'],'dining':['mean'],'museums':['mean'],'art':['mean'],'hiking':['mean'],'gaming':['mean'],'clubbing':['mean'],'reading':['mean'],'tv':['mean'],'theater':['mean'],'movies':['mean'],'concerts':['mean'],'music':['mean'],'shopping':['mean'],'yoga':['mean'], 'exphappy':['mean'], 'attr3_1':['mean'],'sinc3_1':['mean'],
'fun3_1':['mean'],'intel3_1':['mean'], 'amb3_1':['mean'], 'match_es':['mean']})
df2.columns = df2.columns.droplevel(1)

In [57]:
#dropping some NULLS with multiple missing values across columns
df2.dropna(subset=['imprace'], inplace=True)

In [58]:
#filling the rest of the NANs
cols = ["date", "age", "career_c", "exphappy", "attr3_1", "sinc3_1", "match_es", "intel3_1", "fun3_1", "amb3_1"]
df2[cols]=df2[cols].fillna(df2.mode().iloc[0])

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 550
Data columns (total 38 columns):
iid         544 non-null int64
id          544 non-null float64
wave        544 non-null int64
gender      544 non-null int64
from        544 non-null object
age         544 non-null float64
dec_o       544 non-null int64
race        544 non-null float64
imprace     544 non-null float64
imprelig    544 non-null float64
goal        544 non-null float64
date        544 non-null float64
go_out      544 non-null float64
career_c    544 non-null float64
sports      544 non-null float64
tvsports    544 non-null float64
exercise    544 non-null float64
dining      544 non-null float64
museums     544 non-null float64
art         544 non-null float64
hiking      544 non-null float64
gaming      544 non-null float64
clubbing    544 non-null float64
reading     544 non-null float64
tv          544 non-null float64
theater     544 non-null float64
movies      544 non-null float64
concerts    5

In [11]:
#assinging categorical variables

df2['race'] = pd.Categorical(df2['race'])
df2['gender'] = pd.Categorical(df2['gender'])

In [59]:
#checking for datatype 
df2.race.dtype
df2.gender.dtype

dtype('int64')

## create new dummy POPULAR by session and by gender

In [61]:
#define all cutoffs here (half) (70%)

# 10/10 - THIS IS WAVE 1!!
wave_1_cutoff = 10*0.6 

# 16/19
wave_2m_cutoff = 19*0.6 #9
wave_2f_cutoff = 16*0.6 #8

#10/9
wave_3m_cutoff = 9*0.6 #5
wave_3f_cutoff = 10*0.6

#18/18
wave_4_cutoff = 18*0.6 #9

#10/10
wave_5_cutoff = 10*0.6 #5

#5/5
wave_6_cutoff = 5*0.6 #3

#16/16
wave_7_cutoff = 16*0.6 #8

#10/10
wave_8_cutoff = 6 #5

#20/20
wave_9_cutoff = 20*0.6 #10

#9/9
wave_10_cutoff = 9*0.6 #5

#21/21
wave_11_cutoff = 21*0.6 #11

#14/15
wave_12m_cutoff = 15*0.6 #8
wave_12f_cutoff = 14*0.6 #7

#9/10
wave_13m_cutoff = 7 #5
wave_13f_cutoff = 9*0.6

#18/20
wave_14m_cutoff = 20*0.6 #10
wave_14f_cutoff = 18*0.6 #9

#19/18
wave_15m_cutoff = 18*0.6 #9
wave_15f_cutoff = 19*0.6 #10

#8/6
wave_16m_cutoff = 6*0.6 #3
wave_16f_cutoff = 8*0.6 #4

#14/10
wave_17m_cutoff = 10*0.6 #5
wave_17f_cutoff = 14*0.6 #7

#6/6
wave_18_cutoff = 6*0.6 #3

#15/16
wave_19m_cutoff = 16*0.6 #8
wave_19f_cutoff = 15*0.6

#8/6
wave_20m_cutoff = 6*0.6 #3
wave_20f_cutoff = 8*0.6 #4

#22/22
wave_21_cutoff = 22*0.6 #11

In [62]:
#session 1
ses1 = df2[df2['wave'] == 1]
def popularity (row):
    if row['dec_o'] >= wave_1_cutoff :
          return 'popular'
    if row['dec_o'] < wave_1_cutoff :
          return 'no'
ses1.apply(lambda row: popularity(row), axis=1)
ses1['popularity'] = ses1.apply (lambda row: popularity(row), axis=1)
dummies = pd.get_dummies(ses1['popularity'])
ses1 = pd.concat([ses1, dummies], axis=1)
#ses1.drop(['popularity'], inplace=True, axis=1)
ses1.drop(['no'], inplace=True, axis=1)
ses1.drop(['popular'], inplace=True, axis=1)
ses1

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
0,1,1.0,1,0,Chicago,21.0,5,4.0,2.0,4.0,...,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0,no
1,2,2.0,1,0,Alabama,24.0,6,2.0,2.0,5.0,...,3.0,1.0,4.0,7.0,5.0,10.0,8.0,3.0,3.0,popular
2,3,3.0,1,0,Connecticut,25.0,5,2.0,8.0,4.0,...,8.0,7.0,4.0,8.0,9.0,8.0,9.0,8.0,2.0,no
3,4,4.0,1,0,Texas,23.0,6,2.0,1.0,1.0,...,1.0,8.0,1.0,7.0,8.0,9.0,7.0,8.0,2.0,popular
4,5,5.0,1,0,Bowdoin College,21.0,3,2.0,8.0,1.0,...,8.0,3.0,7.0,6.0,3.0,6.0,10.0,8.0,2.0,no
5,6,6.0,1,0,MD,23.0,5,4.0,1.0,1.0,...,1.0,1.0,4.0,5.0,7.0,8.0,9.0,5.0,4.0,no
6,7,7.0,1,0,Southern California,22.0,7,4.0,2.0,4.0,...,10.0,10.0,7.0,6.0,6.0,5.0,7.0,7.0,2.0,popular
7,8,8.0,1,0,"London, England",25.0,8,2.0,1.0,1.0,...,8.0,6.0,6.0,7.0,4.0,8.0,8.0,8.0,2.0,popular
8,9,9.0,1,0,"Palm Springs, California",26.0,7,6.0,1.0,1.0,...,8.0,3.0,6.0,7.0,6.0,10.0,7.0,7.0,1.0,popular
9,10,10.0,1,0,94115,26.0,4,2.0,4.0,4.0,...,7.0,7.0,7.0,6.0,8.0,10.0,6.0,9.0,2.0,no


In [63]:
#session 2 male
ses2m = df2[(df2['wave'] == 2) & (df2['gender'] == 1)]
def popularity (row):
    if row['dec_o'] >= wave_2m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_2m_cutoff :
          return 'no'
ses2m.apply (lambda row: popularity(row), axis=1)
ses2m['popularity'] = ses2m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses2m['popularity'])
ses2m = pd.concat([ses2m, dummies], axis=1)
#ses2m.drop(['popularity'], inplace=True, axis=1)
ses2m.drop(['no'], inplace=True, axis=1)
#ses3f.drop(['popular'], inplace=True, axis=1)
ses2m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
39,40,1.0,2,1,"SF Bay Area, CA",25.0,4,4.0,9.0,6.0,...,4.0,2.0,3.0,3.0,10.0,8.0,9.0,5.0,0.0,no
40,41,2.0,2,1,China,30.0,0,4.0,1.0,1.0,...,5.0,1.0,7.0,5.0,7.0,5.0,8.0,5.0,2.0,no
41,42,3.0,2,1,San Francisco(home)/Los Angeles(undergrad),23.0,4,4.0,1.0,6.0,...,2.0,1.0,6.0,6.0,9.0,6.0,8.0,4.0,1.0,no
42,43,4.0,2,1,Spain,24.0,3,2.0,1.0,1.0,...,2.0,1.0,3.0,3.0,8.0,3.0,7.0,2.0,0.0,no
43,44,5.0,2,1,Israel,24.0,4,2.0,3.0,3.0,...,6.0,8.0,7.0,7.0,9.0,8.0,9.0,9.0,3.0,no
44,45,6.0,2,1,Switzerland,28.0,4,2.0,6.0,6.0,...,6.0,3.0,7.0,7.0,8.0,6.0,8.0,7.0,9.0,no
45,46,7.0,2,1,Italy,26.0,9,2.0,2.0,3.0,...,2.0,7.0,7.0,7.0,8.0,5.0,7.0,7.0,4.0,no
46,47,8.0,2,1,Italy,30.0,1,2.0,2.0,2.0,...,2.0,1.0,8.0,8.0,7.0,9.0,9.0,3.0,5.0,no
47,48,9.0,2,1,Israel,22.0,8,2.0,9.0,2.0,...,6.0,2.0,6.0,7.0,9.0,6.0,7.0,6.0,2.0,no
48,49,10.0,2,1,Philadelphia,25.0,8,4.0,1.0,3.0,...,2.0,7.0,6.0,7.0,10.0,10.0,10.0,8.0,4.0,no


In [64]:
#session 2 female
ses2f = df2[(df2['wave'] == 2) & (df2['gender'] == 0)]
def popularity (row):
    if row['dec_o'] >= wave_2f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_2f_cutoff :
          return 'no'
ses2f.apply (lambda row: popularity(row), axis=1)
ses2f['popularity'] = ses2f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses2f['popularity'])
ses2f = pd.concat([ses2f, dummies], axis=1)
#ses2f.drop(['popularity'], inplace=True, axis=1)
ses2f.drop(['no'], inplace=True, axis=1)
ses2f.drop(['popular'], inplace=True, axis=1)
ses2f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
20,21,1.0,2,0,New Jersey,21.0,3,4.0,8.0,10.0,...,5.0,2.0,5.0,5.0,9.0,7.0,9.0,10.0,0.0,no
21,22,2.0,2,0,"Ann Arbor, MI",27.0,3,1.0,1.0,2.0,...,8.0,8.0,4.0,7.0,8.0,7.0,8.0,7.0,2.0,no
22,23,3.0,2,0,Hong Kong,21.0,6,4.0,9.0,6.0,...,10.0,3.0,5.0,5.0,8.0,6.0,7.0,8.0,3.0,no
23,24,4.0,2,0,new york,27.0,8,2.0,2.0,3.0,...,6.0,7.0,5.0,7.0,9.0,9.0,8.0,8.0,1.0,no
24,25,5.0,2,0,Iowa,25.0,14,2.0,10.0,2.0,...,8.0,4.0,5.0,7.0,9.0,7.0,8.0,6.0,0.0,popular
25,26,6.0,2,0,California,23.0,15,2.0,2.0,4.0,...,5.0,1.0,7.0,7.0,8.0,8.0,8.0,8.0,0.0,popular
26,27,7.0,2,0,"Westchester, NY",22.0,15,3.0,1.0,5.0,...,7.0,8.0,5.0,6.0,6.0,7.0,7.0,7.0,1.0,popular
28,29,9.0,2,0,"Washington, DC",22.0,3,2.0,4.0,5.0,...,9.0,5.0,7.0,6.0,9.0,8.0,7.0,8.0,3.0,no
29,30,10.0,2,0,Boston,23.0,4,2.0,2.0,4.0,...,6.0,4.0,3.0,7.0,8.0,9.0,8.0,8.0,1.0,no
30,31,11.0,2,0,"Washington, DC",24.0,13,2.0,4.0,8.0,...,10.0,6.0,5.0,8.0,8.0,9.0,8.0,9.0,3.0,popular


In [65]:
#session 3 male (4)
ses3m = df2[(df2['wave'] == 3) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_3m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_3m_cutoff :
          return 'no'
ses3m.apply (lambda row: popularity(row), axis=1)
ses3m['popularity'] = ses3m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses3m['popularity'])
ses3m = pd.concat([ses3m, dummies], axis=1)
#ses3m.drop(['popularity'], inplace=True, axis=1)
ses3m.drop(['no'], inplace=True, axis=1)
ses3m.drop(['popular'], inplace=True, axis=1)
ses3m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
65,66,1.0,3,1,Mexico,29.0,3,3.0,1.0,1.0,...,4.0,2.0,5.0,6.0,7.0,7.0,8.0,8.0,3.0,no
66,67,2.0,3,1,"Torrance, CA",22.0,4,6.0,1.0,1.0,...,2.0,1.0,5.0,5.0,7.0,6.0,7.0,7.0,3.0,no
67,68,3.0,3,1,"St. Louis, MO",27.0,2,1.0,1.0,1.0,...,7.0,1.0,5.0,6.0,8.0,7.0,8.0,8.0,2.0,no
68,69,4.0,3,1,"London, UK",28.0,7,2.0,2.0,2.0,...,2.0,1.0,3.0,5.0,9.0,5.0,8.0,8.0,1.0,popular
69,70,5.0,3,1,"Boston, MA",26.0,3,2.0,5.0,1.0,...,5.0,6.0,3.0,8.0,6.0,8.0,9.0,6.0,0.0,no
70,71,6.0,3,1,Minnesota,21.0,4,2.0,6.0,7.0,...,2.0,1.0,9.0,6.0,8.0,8.0,10.0,10.0,2.0,no
71,72,7.0,3,1,"Boston, MA",22.0,2,2.0,8.0,1.0,...,8.0,4.0,7.0,7.0,9.0,8.0,8.0,8.0,1.0,no
72,73,8.0,3,1,Chicago,22.0,4,2.0,7.0,3.0,...,4.0,1.0,2.0,7.0,7.0,8.0,7.0,8.0,0.0,no
73,74,9.0,3,1,Spain,25.0,3,2.0,3.0,1.0,...,3.0,5.0,9.0,7.0,9.0,9.0,8.0,7.0,2.0,no
74,75,10.0,3,1,Philadelphia,24.0,2,6.0,3.0,3.0,...,5.0,5.0,5.0,7.0,7.0,8.0,8.0,8.0,5.0,no


In [66]:
#session 3 female (5)
ses3f = df2[(df2['wave'] == 3) & (df2['gender'] == 1)]
def popularity (row):
    if row['dec_o'] >= wave_3f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_3f_cutoff :
          return 'no'
ses3f.apply (lambda row: popularity(row), axis=1)
ses3f['popularity'] = ses3f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses3f['popularity'])
ses3f = pd.concat([ses3f, dummies], axis=1)
#ses3f.drop(['popularity'], inplace=True, axis=1)
ses3f.drop(['no'], inplace=True, axis=1)
ses3f.drop(['popular'], inplace=True, axis=1)
ses3f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
65,66,1.0,3,1,Mexico,29.0,3,3.0,1.0,1.0,...,4.0,2.0,5.0,6.0,7.0,7.0,8.0,8.0,3.0,no
66,67,2.0,3,1,"Torrance, CA",22.0,4,6.0,1.0,1.0,...,2.0,1.0,5.0,5.0,7.0,6.0,7.0,7.0,3.0,no
67,68,3.0,3,1,"St. Louis, MO",27.0,2,1.0,1.0,1.0,...,7.0,1.0,5.0,6.0,8.0,7.0,8.0,8.0,2.0,no
68,69,4.0,3,1,"London, UK",28.0,7,2.0,2.0,2.0,...,2.0,1.0,3.0,5.0,9.0,5.0,8.0,8.0,1.0,popular
69,70,5.0,3,1,"Boston, MA",26.0,3,2.0,5.0,1.0,...,5.0,6.0,3.0,8.0,6.0,8.0,9.0,6.0,0.0,no
70,71,6.0,3,1,Minnesota,21.0,4,2.0,6.0,7.0,...,2.0,1.0,9.0,6.0,8.0,8.0,10.0,10.0,2.0,no
71,72,7.0,3,1,"Boston, MA",22.0,2,2.0,8.0,1.0,...,8.0,4.0,7.0,7.0,9.0,8.0,8.0,8.0,1.0,no
72,73,8.0,3,1,Chicago,22.0,4,2.0,7.0,3.0,...,4.0,1.0,2.0,7.0,7.0,8.0,7.0,8.0,0.0,no
73,74,9.0,3,1,Spain,25.0,3,2.0,3.0,1.0,...,3.0,5.0,9.0,7.0,9.0,9.0,8.0,7.0,2.0,no
74,75,10.0,3,1,Philadelphia,24.0,2,6.0,3.0,3.0,...,5.0,5.0,5.0,7.0,7.0,8.0,8.0,8.0,5.0,no


In [112]:
#session 4 (9)
ses4 = df2[(df2['wave'] == 4)]

def popularity (row):
    if row['dec_o'] >= 9 :
          return 'popular'
    if row['dec_o'] < 9 :
          return 'no'
ses4.apply (lambda row: popularity(row), axis=1)
ses4['popularity'] = ses4.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses4['popularity'])
ses4 = pd.concat([ses4, dummies], axis=1)
#ses4.drop(['popularity'], inplace=True, axis=1)
ses4.drop(['no'], inplace=True, axis=1)
ses4.drop(['popular'], inplace=True, axis=1)
ses4

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
75,76,1.0,4,0,Ohio,34.0,9,2.0,5.0,7.0,...,7.0,7.0,6.0,6.0,6.0,6.0,8.0,4.0,4.0,popular
76,77,2.0,4,0,Minnesota,22.0,8,2.0,2.0,2.0,...,7.0,9.0,6.0,6.0,8.0,8.0,8.0,8.0,2.0,no
77,78,3.0,4,0,Ohio,23.0,5,3.0,1.0,4.0,...,5.0,10.0,7.0,3.0,10.0,3.0,8.0,8.0,1.0,no
78,79,4.0,4,0,India/Venezuela,26.0,9,4.0,8.0,8.0,...,3.0,3.0,5.0,7.0,9.0,8.0,7.0,7.0,2.0,popular
79,80,5.0,4,0,China,27.0,5,4.0,3.0,1.0,...,9.0,2.0,7.0,8.0,8.0,8.0,8.0,8.0,4.0,no
80,81,6.0,4,0,Texas,28.0,14,2.0,6.0,9.0,...,7.0,4.0,5.0,8.0,7.0,8.0,7.0,7.0,2.0,popular
81,82,7.0,4,0,Washington State,25.0,10,6.0,2.0,7.0,...,6.0,7.0,7.0,8.0,9.0,8.0,8.0,7.0,5.0,popular
82,83,8.0,4,0,Panama,35.0,13,3.0,1.0,5.0,...,4.0,7.0,6.0,5.0,9.0,9.0,5.0,5.0,0.0,popular
83,84,9.0,4,0,spain,32.0,9,3.0,1.0,1.0,...,5.0,5.0,7.0,7.0,9.0,7.0,8.0,5.0,2.0,popular
84,85,10.0,4,0,Arizona,30.0,7,2.0,3.0,5.0,...,5.0,3.0,4.0,7.0,9.0,9.0,8.0,9.0,3.0,no


In [68]:
#session 5 (5) - filled NaN
ses5 = df2[(df2['wave'] == 5)]
ses5m = df2[(df2['wave'] == 5) & (df2['gender'] == 1)]
cols = ["age"]
ses5[cols]=ses5[cols].fillna(ses5m.mode().iloc[0])

def popularity (row):
    if row['dec_o'] >= wave_5_cutoff :
          return 'popular'
    if row['dec_o'] < wave_5_cutoff :
          return 'no'
ses5.apply (lambda row: popularity(row), axis=1)
ses5['popularity'] = ses5.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses5['popularity'])
ses5 = pd.concat([ses5, dummies], axis=1)
#ses5.drop(['popularity'], inplace=True, axis=1)
ses5.drop(['no'], inplace=True, axis=1)
ses5.drop(['popular'], inplace=True, axis=1)
ses5

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
111,112,1.0,5,0,New Jersey,20.0,8,6.0,1.0,1.0,...,4.0,6.0,4.0,9.0,10.0,9.0,9.0,9.0,8.0,popular
112,113,2.0,5,0,"Milwaukee, Wisconsin",21.0,4,2.0,3.0,8.0,...,10.0,7.0,7.0,8.0,10.0,10.0,8.0,9.0,0.0,no
113,114,3.0,5,0,New York,20.0,2,1.0,2.0,4.0,...,2.0,2.0,6.0,7.0,9.0,8.0,6.0,7.0,3.0,no
114,115,4.0,5,0,brooklyn ny,21.0,5,2.0,10.0,7.0,...,8.0,1.0,3.0,8.0,7.0,8.0,9.0,8.0,3.0,no
115,116,5.0,5,0,France,21.0,6,2.0,4.0,1.0,...,10.0,8.0,7.0,8.0,9.0,8.0,9.0,9.0,4.0,popular
116,117,6.0,5,0,"Long Island, NY",21.0,5,2.0,8.0,10.0,...,9.0,3.0,6.0,8.0,7.0,8.0,8.0,8.0,1.0,no
117,119,10.0,5,0,Puerto Rico,19.0,8,3.0,5.0,2.0,...,7.0,8.0,4.0,8.0,10.0,10.0,10.0,10.0,5.0,popular
118,120,11.0,5,0,"Long Island, NY",20.0,7,3.0,1.0,4.0,...,3.0,5.0,3.0,6.0,8.0,8.0,8.0,9.0,2.0,popular
119,121,12.0,5,0,"Memphis, TN",21.0,2,2.0,8.0,1.0,...,10.0,4.0,2.0,2.0,9.0,9.0,7.0,9.0,2.0,no
120,122,1.0,5,1,Long Island,22.0,7,6.0,1.0,1.0,...,5.0,4.0,6.0,7.0,10.0,8.0,8.0,8.0,4.0,popular


In [69]:
#session 6 (3) 
ses6 = df2[(df2['wave'] == 6)]
#ses6f = df2[(df2['wave'] == 6) & (df2['gender'] == 0)]
#cols = ["age", "race"]
#ses6[cols]=ses6[cols].fillna(ses6f.mode().iloc[0])

def popularity (row):
    if row['dec_o'] >= wave_6_cutoff :
          return 'popular'
    if row['dec_o'] < wave_6_cutoff :
          return 'no'
ses6.apply (lambda row: popularity(row), axis=1)
ses6['popularity'] = ses6.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses6['popularity'])
ses6 = pd.concat([ses6, dummies], axis=1)
#ses6.drop(['popularity'], inplace=True, axis=1)
ses6.drop(['no'], inplace=True, axis=1)
ses6.drop(['popular'], inplace=True, axis=1)
ses6

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
130,132,1.0,6,0,Los Angeles,27.0,5,2.0,10.0,1.0,...,1.0,10.0,5.0,8.0,10.0,10.0,9.0,10.0,0.0,popular
131,133,2.0,6,0,china,24.0,2,4.0,8.0,6.0,...,7.0,6.0,5.0,6.0,8.0,7.0,8.0,7.0,2.0,no
132,134,3.0,6,0,Pougkeepsie NY,23.0,3,2.0,8.0,10.0,...,8.0,8.0,8.0,7.0,8.0,8.0,8.0,8.0,1.0,popular
133,135,5.0,6,0,"sofia, bg",26.0,3,2.0,9.0,6.0,...,2.0,9.0,6.0,9.0,10.0,7.0,9.0,6.0,2.0,popular
135,137,1.0,6,1,New York,26.0,1,4.0,1.0,1.0,...,8.0,3.0,7.0,7.0,9.0,8.0,9.0,8.0,2.0,no
136,138,2.0,6,1,"Greenwich, CT",32.0,3,2.0,9.0,6.0,...,6.0,2.0,6.0,8.0,8.0,8.0,8.0,8.0,3.0,popular
137,139,4.0,6,1,Taiwan,37.0,1,4.0,6.0,1.0,...,7.0,8.0,8.0,8.0,8.0,9.0,9.0,6.0,2.0,no
138,140,6.0,6,1,Europe,29.0,3,2.0,5.0,5.0,...,4.0,3.0,6.0,7.0,7.0,9.0,8.0,8.0,2.0,popular
139,141,10.0,6,1,Colombia,28.0,1,3.0,4.0,4.0,...,6.0,6.0,4.0,7.0,9.0,8.0,8.0,6.0,2.0,no


In [70]:
#session 7 (8)
ses7 = df2[(df2['wave'] == 7)]

def popularity (row):
    if row['dec_o'] >= wave_7_cutoff :
          return 'popular'
    if row['dec_o'] < wave_7_cutoff :
          return 'no'
ses7.apply (lambda row: popularity(row), axis=1)
ses7['popularity'] = ses7.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses7['popularity'])
ses7 = pd.concat([ses7, dummies], axis=1)
#ses7.drop(['popularity'], inplace=True, axis=1)
ses7.drop(['no'], inplace=True, axis=1)
ses7.drop(['popular'], inplace=True, axis=1)
ses7

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
140,142,1.0,7,0,california,24.0,9,2.0,1.0,4.0,...,6.0,6.0,7.0,9.0,9.0,9.0,10.0,7.0,2.0,no
141,143,2.0,7,0,Albania,33.0,1,2.0,6.0,2.0,...,6.0,4.0,6.0,7.0,10.0,7.0,8.0,7.0,3.0,no
142,144,3.0,7,0,PA,23.0,6,2.0,5.0,2.0,...,7.0,4.0,5.0,7.0,8.0,8.0,9.0,9.0,2.0,no
143,145,4.0,7,0,Minnesota,24.0,5,2.0,9.0,7.0,...,9.0,2.0,5.0,5.0,5.0,7.0,9.0,7.0,2.0,no
144,146,5.0,7,0,New York,36.0,5,2.0,1.0,7.0,...,8.0,3.0,5.0,8.0,8.0,8.0,8.0,8.0,2.0,no
145,147,6.0,7,0,New York,26.0,8,4.0,3.0,3.0,...,7.0,10.0,5.0,8.0,7.0,8.0,9.0,7.0,7.0,no
146,148,7.0,7,0,"Cherry Hill, NJ",27.0,11,4.0,7.0,4.0,...,9.0,7.0,5.0,8.0,9.0,9.0,9.0,8.0,2.0,popular
147,149,8.0,7,0,Minneapolis,28.0,12,2.0,3.0,3.0,...,9.0,8.0,4.0,8.0,8.0,8.0,8.0,8.0,5.0,popular
148,150,9.0,7,0,NYC,31.0,11,2.0,3.0,3.0,...,2.0,10.0,5.0,7.0,9.0,8.0,9.0,8.0,1.0,popular
149,151,10.0,7,0,"ottawa, canada",28.0,14,2.0,1.0,2.0,...,7.0,7.0,3.0,9.0,10.0,10.0,10.0,8.0,1.0,popular


In [71]:
#session 8 (5)
ses8 = df2[(df2['wave'] == 8)]

def popularity (row):
    if row['dec_o'] >= wave_8_cutoff :
          return 'popular'
    if row['dec_o'] < wave_8_cutoff :
          return 'no'
ses8.apply (lambda row: popularity(row), axis=1)
ses8['popularity'] = ses8.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses8['popularity'])
ses8 = pd.concat([ses8, dummies], axis=1)
#ses8.drop(['popularity'], inplace=True, axis=1)
ses8.drop(['no'], inplace=True, axis=1)
ses8.drop(['popular'], inplace=True, axis=1)
ses8

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
172,174,1.0,8,0,Pennsylvania,29.0,2,4.0,1.0,1.0,...,6.0,5.0,5.0,9.0,10.0,8.0,8.0,6.0,4.0,no
173,175,2.0,8,0,Ohio,23.0,4,6.0,5.0,8.0,...,5.0,3.0,7.0,7.0,10.0,8.0,8.0,9.0,2.0,no
174,176,3.0,8,0,California,28.0,4,4.0,10.0,1.0,...,6.0,3.0,7.0,6.0,9.0,7.0,8.0,6.0,4.0,no
175,177,4.0,8,0,"San Diego, CA",22.0,0,2.0,8.0,3.0,...,5.0,1.0,6.0,9.0,9.0,8.0,9.0,9.0,2.0,no
176,178,5.0,8,0,New York,35.0,3,4.0,5.0,5.0,...,6.0,8.0,5.0,9.0,9.0,7.0,8.0,7.0,4.0,no
177,179,6.0,8,0,California,30.0,8,3.0,5.0,1.0,...,10.0,9.0,2.0,8.0,10.0,6.0,8.0,10.0,1.0,popular
178,180,7.0,8,0,taiwan,24.0,9,4.0,2.0,2.0,...,9.0,7.0,6.0,9.0,9.0,10.0,9.0,9.0,2.0,popular
179,181,8.0,8,0,"Brooklyn, NY",22.0,6,1.0,7.0,8.0,...,9.0,6.0,4.0,9.0,9.0,9.0,9.0,9.0,2.0,popular
180,182,9.0,8,0,New Jersey,22.0,2,1.0,7.0,5.0,...,8.0,1.0,6.0,8.0,8.0,8.0,8.0,10.0,2.0,no
181,183,10.0,8,0,Uzbekistan,24.0,2,2.0,1.0,1.0,...,8.0,1.0,5.0,9.0,7.0,9.0,9.0,7.0,5.0,no


In [72]:
#session 9 (10)
ses9 = df2[(df2['wave'] == 9)]

def popularity (row):
    if row['dec_o'] >= wave_9_cutoff :
          return 'popular'
    if row['dec_o'] < wave_9_cutoff :
          return 'no'
ses9.apply (lambda row: popularity(row), axis=1)
ses9['popularity'] = ses9.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses9['popularity'])
ses9 = pd.concat([ses9, dummies], axis=1)
#ses9.drop(['popularity'], inplace=True, axis=1)
ses9.drop(['no'], inplace=True, axis=1)
ses9.drop(['popular'], inplace=True, axis=1)
ses9

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
192,194,1.0,9,0,New York,24.0,3,2.0,9.0,9.0,...,9.0,4.0,7.0,9.0,10.0,8.0,10.0,10.0,1.0,no
193,195,2.0,9,0,"Rochester, NY",27.0,16,2.0,3.0,9.0,...,5.0,1.0,8.0,9.0,8.0,10.0,10.0,8.0,6.0,popular
194,196,3.0,9,0,"Taipei, Taiwan",25.0,8,4.0,1.0,6.0,...,7.0,9.0,6.0,9.0,9.0,9.0,9.0,9.0,2.0,no
195,197,4.0,9,0,Berkeley,24.0,16,6.0,2.0,1.0,...,7.0,3.0,3.0,7.0,8.0,9.0,9.0,7.0,7.0,popular
196,198,5.0,9,0,China,23.0,3,4.0,8.0,5.0,...,10.0,4.0,6.0,7.0,9.0,7.0,9.0,7.0,0.0,no
197,199,6.0,9,0,New York,30.0,15,2.0,2.0,1.0,...,7.0,1.0,4.0,9.0,7.0,7.0,9.0,9.0,2.0,popular
198,200,7.0,9,0,Northern California,25.0,17,2.0,1.0,1.0,...,3.0,4.0,5.0,8.0,10.0,8.0,8.0,8.0,2.0,popular
199,201,8.0,9,0,New York,25.0,7,2.0,8.0,8.0,...,7.0,6.0,2.0,7.0,8.0,9.0,9.0,8.0,1.0,no
200,202,9.0,9,0,"Silver Spring, MD",22.0,11,2.0,8.0,5.0,...,7.0,1.0,3.0,7.0,10.0,9.0,10.0,8.0,3.0,no
201,203,10.0,9,0,Vestal,22.0,15,2.0,4.0,10.0,...,9.0,2.0,3.0,9.0,8.0,9.0,10.0,7.0,2.0,popular


In [73]:
#session 10 (5)
ses10 = df2[(df2['wave'] == 10)]

def popularity (row):
    if row['dec_o'] >= wave_10_cutoff :
          return 'popular'
    if row['dec_o'] < wave_10_cutoff :
          return 'no'
ses10.apply (lambda row: popularity(row), axis=1)
ses10['popularity'] = ses10.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses10['popularity'])
ses10 = pd.concat([ses10, dummies], axis=1)
#ses10.drop(['popularity'], inplace=True, axis=1)
ses10.drop(['no'], inplace=True, axis=1)
ses10.drop(['popular'], inplace=True, axis=1)
ses10

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
232,234,1.0,10,0,Virginia,22.0,4,6.0,1.0,5.0,...,7.0,4.0,5.0,7.0,7.0,8.0,8.0,10.0,1.0,no
233,235,2.0,10,0,"Boston, MA",23.0,8,2.0,3.0,1.0,...,10.0,7.0,5.0,6.0,9.0,9.0,8.0,9.0,1.0,popular
234,236,3.0,10,0,Indiana,24.0,1,4.0,3.0,3.0,...,4.0,2.0,7.0,7.0,9.0,10.0,8.0,8.0,4.0,no
235,237,4.0,10,0,lOS aNGELES,34.0,3,2.0,10.0,1.0,...,2.0,10.0,4.0,8.0,10.0,7.0,8.0,8.0,0.0,no
236,238,5.0,10,0,Wisconsin,27.0,5,2.0,3.0,5.0,...,7.0,6.0,5.0,10.0,9.0,10.0,9.0,8.0,2.0,no
237,239,6.0,10,0,"Born in Montana, raised in South Jersey (nr. P...",34.0,3,2.0,6.0,6.0,...,1.0,8.0,7.0,7.0,9.0,7.0,9.0,4.0,1.0,no
238,240,7.0,10,0,New York,26.0,6,4.0,1.0,1.0,...,10.0,3.0,2.0,3.0,8.0,5.0,3.0,6.0,1.0,popular
239,241,8.0,10,0,New York,29.0,8,4.0,1.0,1.0,...,6.0,3.0,2.0,8.0,10.0,8.0,8.0,8.0,2.0,popular
240,242,9.0,10,0,uruguay,26.0,7,3.0,3.0,3.0,...,1.0,8.0,6.0,7.0,9.0,7.0,8.0,4.0,3.0,popular
241,243,1.0,10,1,"Portland, Oregon",27.0,7,2.0,2.0,3.0,...,6.0,2.0,6.0,7.0,9.0,8.0,9.0,5.0,3.0,popular


In [74]:
#session 11 (10) 
ses11 = df2[(df2['wave'] == 11)]

def popularity (row):
    if row['dec_o'] >= wave_11_cutoff :
          return 'popular'
    if row['dec_o'] < wave_11_cutoff :
          return 'no'
ses11.apply (lambda row: popularity(row), axis=1)
ses11['popularity'] = ses11.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses11['popularity'])
ses11 = pd.concat([ses11, dummies], axis=1)
#ses11.drop(['popularity'], inplace=True, axis=1)
ses11.drop(['no'], inplace=True, axis=1)
ses11.drop(['popular'], inplace=True, axis=1)
ses11

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
250,252,1.0,11,0,Mexico,28.0,11,3.0,1.0,1.0,...,9.0,2.0,10.0,9.0,9.0,9.0,9.0,10.0,2.0,no
251,253,2.0,11,0,Japan,33.0,5,4.0,1.0,1.0,...,7.0,3.0,7.0,8.0,10.0,9.0,8.0,7.0,8.0,no
252,254,3.0,11,0,"Boulder, Colorado",22.0,9,2.0,8.0,3.0,...,7.0,8.0,6.0,8.0,7.0,8.0,10.0,8.0,2.0,no
253,255,4.0,11,0,Maryland,26.0,7,4.0,3.0,1.0,...,9.0,1.0,6.0,6.0,8.0,6.0,7.0,6.0,0.0,no
254,256,5.0,11,0,Northern Virginia,24.0,16,4.0,6.0,5.0,...,9.0,3.0,6.0,6.0,9.0,6.0,8.0,6.0,2.0,popular
255,257,6.0,11,0,I am from NYC,24.0,11,2.0,5.0,6.0,...,4.0,10.0,3.0,7.0,9.0,7.0,9.0,10.0,0.0,no
256,258,7.0,11,0,Seattle,25.0,14,6.0,1.0,10.0,...,6.0,2.0,6.0,6.0,10.0,8.0,9.0,5.0,5.0,popular
257,259,8.0,11,0,Texas,25.0,7,2.0,2.0,2.0,...,5.0,6.0,5.0,7.0,9.0,8.0,8.0,9.0,2.0,no
258,260,9.0,11,0,"Malaysia, then Massachusetts",27.0,10,4.0,4.0,4.0,...,5.0,3.0,6.0,6.0,9.0,7.0,9.0,6.0,4.0,no
259,261,10.0,11,0,"Katonah, NY (more recently, Boston)",26.0,9,4.0,1.0,4.0,...,4.0,6.0,3.0,5.0,8.0,7.0,7.0,8.0,4.0,no


In [108]:
#session 12 male (7)
ses12m = df2[(df2['wave'] == 12) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_12m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_12m_cutoff :
          return 'no'
ses12m.apply (lambda row: popularity(row), axis=1)
ses12m['popularity'] = ses12m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses12m['popularity'])
ses12m = pd.concat([ses12m, dummies], axis=1)
#ses12m.drop(['popularity'], inplace=True, axis=1)
ses12m.drop(['no'], inplace=True, axis=1)
ses12m.drop(['popular'], inplace=True, axis=1)
ses12m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
306,308,1.0,12,1,"Cambridge, Massachusetts",27.0,1,4.0,7.0,3.0,...,7.0,7.0,7.0,7.0,10.0,8.0,8.0,7.0,2.0,no
307,309,2.0,12,1,NYC,32.0,7,2.0,5.0,9.0,...,4.0,4.0,7.0,7.0,8.0,9.0,9.0,7.0,5.0,no
308,310,3.0,12,1,NJ,25.0,3,2.0,7.0,8.0,...,5.0,3.0,6.0,7.0,6.0,8.0,7.0,4.0,2.0,no
309,311,4.0,12,1,New York,29.0,8,4.0,1.0,1.0,...,9.0,8.0,8.0,9.0,9.0,10.0,9.0,3.0,4.0,no
310,312,5.0,12,1,NYC,23.0,7,2.0,3.0,1.0,...,2.0,1.0,5.0,8.0,7.0,7.0,9.0,9.0,2.0,no
311,313,6.0,12,1,Boston,27.0,9,2.0,2.0,3.0,...,5.0,1.0,6.0,7.0,8.0,8.0,8.0,6.0,2.0,popular
312,314,7.0,12,1,"New Delhi, India",32.0,1,6.0,3.0,1.0,...,8.0,8.0,6.0,8.0,8.0,8.0,9.0,7.0,2.0,no
313,315,8.0,12,1,way too little space here. world citizen.,27.0,5,2.0,1.0,4.0,...,2.0,2.0,5.0,6.0,10.0,6.0,8.0,7.0,1.0,no
314,316,9.0,12,1,Philadelphia,25.0,9,2.0,3.0,3.0,...,4.0,1.0,5.0,6.0,8.0,7.0,8.0,9.0,4.0,popular
315,317,10.0,12,1,Italy,30.0,9,2.0,1.0,1.0,...,1.0,1.0,7.0,7.0,3.0,8.0,8.0,5.0,5.0,popular


In [102]:
#session 12 female (7)
ses12f = df2[(df2['wave'] == 12) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_12f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_12f_cutoff :
          return 'no'
ses12f.apply (lambda row: popularity(row), axis=1)
ses12f['popularity'] = ses12f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses12f['popularity'])
ses12f = pd.concat([ses12f, dummies], axis=1)
#ses12f.drop(['popularity'], inplace=True, axis=1)
ses12f.drop(['no'], inplace=True, axis=1)
ses12f.drop(['popular'], inplace=True, axis=1)
ses12f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
292,294,1.0,12,0,New York City,30.0,9,4.0,1.0,1.0,...,8.0,5.0,6.0,9.0,10.0,7.0,9.0,7.0,1.0,popular
293,295,2.0,12,0,Arizona,30.0,2,2.0,5.0,7.0,...,5.0,8.0,6.0,7.0,10.0,8.0,8.0,8.0,2.0,no
294,296,3.0,12,0,New York,28.0,5,2.0,10.0,10.0,...,9.0,7.0,3.0,8.0,9.0,6.0,10.0,9.0,4.0,no
295,297,4.0,12,0,Texas,21.0,8,2.0,9.0,8.0,...,8.0,6.0,7.0,8.0,10.0,9.0,8.0,9.0,9.0,no
296,298,5.0,12,0,"Buffalo, NY",22.0,3,2.0,7.0,1.0,...,9.0,7.0,1.0,7.0,10.0,9.0,10.0,10.0,3.0,no
297,299,6.0,12,0,Maryland,24.0,5,4.0,1.0,1.0,...,5.0,4.0,5.0,6.0,9.0,9.0,8.0,9.0,2.0,no
298,300,7.0,12,0,"Buffalo, NY",35.0,3,2.0,1.0,1.0,...,4.0,2.0,9.0,8.0,9.0,9.0,9.0,10.0,6.0,no
299,301,8.0,12,0,"Washington, DC",34.0,4,2.0,3.0,8.0,...,5.0,9.0,2.0,9.0,9.0,8.0,9.0,8.0,4.0,no
300,302,10.0,12,0,Long Island,21.0,0,2.0,8.0,7.0,...,6.0,6.0,8.0,8.0,8.0,8.0,8.0,10.0,4.0,no
301,303,11.0,12,0,"Brooklyn, NY",33.0,8,2.0,2.0,1.0,...,5.0,1.0,6.0,8.0,10.0,9.0,8.0,6.0,1.0,no


In [77]:
#session 13 male (5)
ses13m = df2[(df2['wave'] == 13) & (df2['gender'] == 1)]
#cols = ["age", "race"]
#ses13m[cols]=ses13m[cols].fillna(ses13m.mode().iloc[0])

def popularity (row):
    if row['dec_o'] >= wave_13m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_13m_cutoff :
          return 'no'
ses13m.apply (lambda row: popularity(row), axis=1)
ses13m['popularity'] = ses13m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses13m['popularity'])
ses13m = pd.concat([ses13m, dummies], axis=1)
#ses13m.drop(['popularity'], inplace=True, axis=1)
ses13m.drop(['no'], inplace=True, axis=1)
ses13m.drop(['popular'], inplace=True, axis=1)
ses13m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
330,332,1.0,13,1,japan,30.0,4,4.0,1.0,1.0,...,7.0,1.0,7.0,5.0,10.0,6.0,6.0,10.0,3.0,no
331,333,2.0,13,1,"Portland, OR",23.0,8,2.0,5.0,3.0,...,3.0,6.0,7.0,7.0,4.0,7.0,9.0,9.0,2.0,popular
332,334,3.0,13,1,New York/South Korea/Japan,29.0,5,4.0,1.0,1.0,...,8.0,5.0,5.0,8.0,7.0,8.0,8.0,9.0,1.0,no
333,335,4.0,13,1,Michigan,24.0,6,2.0,1.0,8.0,...,5.0,7.0,6.0,8.0,7.0,9.0,8.0,8.0,1.0,no
334,336,5.0,13,1,Miami,25.0,4,3.0,1.0,5.0,...,3.0,4.0,4.0,5.0,9.0,7.0,7.0,9.0,0.0,no
335,337,6.0,13,1,"Dallas, Texas",28.0,4,2.0,1.0,1.0,...,4.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0,5.0,no
336,338,7.0,13,1,"Bombay, India",27.0,3,6.0,1.0,1.0,...,5.0,10.0,2.0,7.0,9.0,7.0,9.0,10.0,4.0,no


In [78]:
#session 13 female (4)
ses13f = df2[(df2['wave'] == 13) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_13f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_13f_cutoff :
          return 'no'
ses13f.apply (lambda row: popularity(row), axis=1)
ses13f['popularity'] = ses13f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses13f['popularity'])
ses13f = pd.concat([ses13f, dummies], axis=1)
#ses13f.drop(['popularity'], inplace=True, axis=1)
ses13f.drop(['no'], inplace=True, axis=1)
ses13f.drop(['popular'], inplace=True, axis=1)
ses13f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
320,322,1.0,13,0,Tokyo and Texas,34.0,2,4.0,2.0,1.0,...,3.0,1.0,9.0,5.0,9.0,8.0,7.0,7.0,2.0,no
321,323,2.0,13,0,Taiwan,25.0,5,4.0,1.0,1.0,...,8.0,6.0,8.0,7.0,9.0,6.0,8.0,3.0,2.0,no
322,324,3.0,13,0,"Erie, PA",23.0,4,2.0,8.0,3.0,...,7.0,7.0,5.0,9.0,9.0,9.0,9.0,8.0,0.0,no
323,325,4.0,13,0,New York,30.0,3,3.0,2.0,4.0,...,4.0,6.0,6.0,7.0,9.0,7.0,8.0,8.0,2.0,no
324,326,5.0,13,0,"Westchester, new York",36.0,2,1.0,5.0,5.0,...,10.0,8.0,8.0,8.0,9.0,6.0,9.0,8.0,5.0,no
325,327,6.0,13,0,USA/American,29.0,8,2.0,2.0,7.0,...,1.0,5.0,5.0,7.0,7.0,9.0,8.0,10.0,0.0,popular
326,328,7.0,13,0,NYC,30.0,5,2.0,1.0,1.0,...,5.0,10.0,6.0,7.0,10.0,8.0,10.0,10.0,2.0,no
327,329,8.0,13,0,"Great Neck, NY",29.0,3,2.0,1.0,1.0,...,2.0,1.0,4.0,8.0,9.0,5.0,8.0,8.0,1.0,no
328,330,9.0,13,0,California (West Coast),22.0,5,4.0,3.0,3.0,...,5.0,2.0,7.0,8.0,10.0,9.0,7.0,9.0,2.0,no
329,331,10.0,13,0,New Jersey,34.0,1,2.0,1.0,1.0,...,4.0,10.0,4.0,7.0,9.0,9.0,7.0,4.0,2.0,no


In [79]:
#session 14 male (10)
ses14m = df2[(df2['wave'] == 14) & (df2['gender'] == 1)]


def popularity (row):
    if row['dec_o'] >= wave_14m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_14m_cutoff :
          return 'no'
ses14m.apply (lambda row: popularity(row), axis=1)
ses14m['popularity'] = ses14m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses14m['popularity'])
ses14m = pd.concat([ses14m, dummies], axis=1)
#ses14m.drop(['popularity'], inplace=True, axis=1)
ses14m.drop(['no'], inplace=True, axis=1)
ses14m.drop(['popular'], inplace=True, axis=1)
ses14m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
359,361,1.0,14,1,Bronx Science,34.0,4,2.0,5.0,5.0,...,3.0,2.0,5.0,7.0,7.0,5.0,7.0,9.0,4.0,no
360,362,2.0,14,1,Spain,30.0,9,2.0,1.0,1.0,...,7.0,2.0,9.0,7.0,7.0,9.0,8.0,8.0,7.0,no
361,363,3.0,14,1,"nashville, TN",23.0,5,4.0,1.0,1.0,...,8.0,5.0,7.0,5.0,7.0,8.0,8.0,3.0,4.0,no
362,364,4.0,14,1,Miami,27.0,9,3.0,6.0,2.0,...,8.0,1.0,6.0,8.0,8.0,8.0,9.0,9.0,5.0,no
363,365,5.0,14,1,France,30.0,10,2.0,1.0,1.0,...,5.0,5.0,8.0,8.0,2.0,9.0,9.0,9.0,5.0,no
364,366,6.0,14,1,Australia,27.0,18,2.0,4.0,6.0,...,8.0,6.0,5.0,6.0,8.0,9.0,8.0,10.0,5.0,popular
365,367,7.0,14,1,"Toronto, Canada",27.0,12,2.0,6.0,3.0,...,7.0,2.0,8.0,7.0,7.0,8.0,7.0,5.0,3.0,popular
366,368,8.0,14,1,UK/Turkey,24.0,11,6.0,1.0,1.0,...,8.0,4.0,4.0,6.0,8.0,8.0,8.0,7.0,3.0,no
367,369,9.0,14,1,"Boston, MA",25.0,10,2.0,1.0,8.0,...,1.0,1.0,7.0,6.0,9.0,9.0,10.0,7.0,5.0,no
368,370,10.0,14,1,New York,28.0,14,2.0,7.0,4.0,...,4.0,1.0,7.0,8.0,6.0,9.0,9.0,9.0,2.0,popular


In [80]:
#session 14 female (9)
ses14f = df2[(df2['wave'] == 14) & (df2['gender'] == 0)]
#cols = ["age", "race"]
#ses14f[cols]=ses14f[cols].fillna(ses14f.mode().iloc[0])

def popularity (row):
    if row['dec_o'] >= wave_14f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_14f_cutoff :
          return 'no'
ses14f.apply (lambda row: popularity(row), axis=1)
ses14f['popularity'] = ses14f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses14f['popularity'])
ses14f = pd.concat([ses14f, dummies], axis=1)
#ses14f.drop(['popularity'], inplace=True, axis=1)
ses14f.drop(['no'], inplace=True, axis=1)
ses14f.drop(['popular'], inplace=True, axis=1)
ses14f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
339,341,1.0,14,0,New Jersey,23.0,7,4.0,3.0,10.0,...,9.0,10.0,5.0,7.0,9.0,9.0,7.0,9.0,10.0,no
340,342,2.0,14,0,"Palo Alto, CA",33.0,3,3.0,2.0,1.0,...,10.0,4.0,2.0,10.0,10.0,10.0,10.0,10.0,6.0,no
341,343,3.0,14,0,New York,32.0,3,2.0,5.0,6.0,...,9.0,7.0,5.0,4.0,8.0,6.0,10.0,7.0,2.0,no
342,344,4.0,14,0,New Mexico,26.0,7,3.0,1.0,3.0,...,9.0,5.0,5.0,7.0,7.0,8.0,8.0,9.0,9.0,no
343,345,5.0,14,0,Brooklyn,24.0,6,2.0,7.0,6.0,...,6.0,9.0,6.0,7.0,9.0,7.0,9.0,7.0,2.0,no
345,347,7.0,14,0,"WASHINGTON, D.C.",26.0,1,1.0,1.0,1.0,...,10.0,1.0,5.0,8.0,6.0,10.0,10.0,10.0,2.0,no
346,348,8.0,14,0,CALIFORNIA,21.0,1,4.0,3.0,3.0,...,10.0,5.0,5.0,6.0,10.0,10.0,10.0,10.0,3.0,no
347,349,9.0,14,0,New York,33.0,12,2.0,1.0,7.0,...,6.0,7.0,6.0,7.0,8.0,8.0,9.0,7.0,5.0,popular
348,350,10.0,14,0,New Jersey,31.0,13,2.0,9.0,5.0,...,5.0,7.0,6.0,7.0,10.0,6.0,8.0,8.0,2.0,popular
349,351,11.0,14,0,Taiwan,35.0,6,4.0,1.0,3.0,...,5.0,5.0,8.0,8.0,10.0,7.0,8.0,9.0,2.0,no


In [81]:
#session 15 male (9)
ses15m = df2[(df2['wave'] == 15) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_15m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_15m_cutoff :
          return 'no'
ses15m.apply (lambda row: popularity(row), axis=1)
ses15m['popularity'] = ses15m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses15m['popularity'])
ses15m = pd.concat([ses15m, dummies], axis=1)
#ses15m.drop(['popularity'], inplace=True, axis=1)
ses15m.drop(['no'], inplace=True, axis=1)
ses15m.drop(['popular'], inplace=True, axis=1)
ses15m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
395,397,1.0,15,1,New Jersey,24.0,6,2.0,3.0,2.0,...,5.0,1.0,6.0,7.0,7.0,7.0,10.0,9.0,7.0,no
396,398,2.0,15,1,Italy,29.0,12,2.0,7.0,2.0,...,6.0,2.0,8.0,7.0,6.0,10.0,7.0,8.0,3.0,popular
397,399,3.0,15,1,Chicago,29.0,6,2.0,1.0,1.0,...,3.0,8.0,7.0,9.0,9.0,9.0,9.0,9.0,2.0,no
398,400,4.0,15,1,Italy,26.0,14,2.0,7.0,7.0,...,1.0,1.0,3.0,7.0,10.0,6.0,8.0,7.0,8.0,popular
399,401,5.0,15,1,Germany,31.0,9,2.0,1.0,1.0,...,7.0,3.0,9.0,8.0,5.0,7.0,10.0,5.0,6.0,no
400,402,6.0,15,1,Korea,29.0,2,4.0,1.0,1.0,...,6.0,9.0,6.0,6.0,8.0,7.0,10.0,9.0,1.0,no
401,403,7.0,15,1,Paris,22.0,3,2.0,6.0,6.0,...,2.0,1.0,6.0,4.0,8.0,7.0,9.0,9.0,3.0,no
402,404,8.0,15,1,France,23.0,11,2.0,1.0,4.0,...,6.0,3.0,7.0,6.0,9.0,7.0,8.0,8.0,6.0,popular
403,405,9.0,15,1,New York City,30.0,5,1.0,1.0,1.0,...,9.0,5.0,6.0,8.0,8.0,10.0,9.0,10.0,1.0,no
404,406,10.0,15,1,Minneapolis,27.0,6,4.0,1.0,1.0,...,2.0,1.0,6.0,6.0,7.0,8.0,8.0,9.0,2.0,no


In [82]:
#session 15 female
ses15f = df2[(df2['wave'] == 15) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_15f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_15f_cutoff :
          return 'no'
ses15f.apply (lambda row: popularity(row), axis=1)
ses15f['popularity'] = ses15f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses15f['popularity'])
ses15f = pd.concat([ses15f, dummies], axis=1)
#ses15f.drop(['popularity'], inplace=True, axis=1)
ses15f.drop(['no'], inplace=True, axis=1)
ses15f.drop(['popular'], inplace=True, axis=1)
ses15f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
377,379,1.0,15,0,"NYC, San Francisco",38.0,10,2.0,4.0,6.0,...,3.0,5.0,5.0,6.0,9.0,6.0,8.0,9.0,1.0,no
378,380,2.0,15,0,Chicago,33.0,5,1.0,4.0,8.0,...,9.0,9.0,6.0,4.0,7.0,9.0,9.0,9.0,5.0,no
379,381,3.0,15,0,California,24.0,8,2.0,6.0,3.0,...,8.0,7.0,2.0,7.0,7.0,10.0,9.0,8.0,3.0,no
380,382,4.0,15,0,New York City,28.0,8,2.0,8.0,1.0,...,9.0,1.0,6.0,7.0,7.0,7.0,7.0,10.0,3.0,no
381,383,5.0,15,0,Massachusetts,27.0,11,2.0,10.0,5.0,...,7.0,4.0,4.0,7.0,9.0,7.0,6.0,6.0,2.0,no
382,384,6.0,15,0,Connecticut,23.0,13,2.0,9.0,6.0,...,6.0,3.0,8.0,8.0,8.0,9.0,8.0,9.0,10.0,popular
383,385,7.0,15,0,Oregon,25.0,10,2.0,5.0,8.0,...,9.0,8.0,7.0,8.0,8.0,6.0,8.0,6.0,2.0,no
384,386,8.0,15,0,New York,26.0,6,2.0,4.0,2.0,...,8.0,3.0,4.0,7.0,8.0,9.0,9.0,9.0,4.0,no
385,387,9.0,15,0,New Jersey,25.0,14,2.0,5.0,2.0,...,8.0,2.0,6.0,8.0,8.0,8.0,8.0,8.0,8.0,popular
386,388,10.0,15,0,"San Francisco, CA",24.0,9,2.0,1.0,6.0,...,6.0,4.0,4.0,8.0,8.0,7.0,8.0,8.0,1.5,no


In [83]:
#session 16 male (3)
ses16m = df2[(df2['wave'] == 16) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_16m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_16m_cutoff :
          return 'no'
ses16m.apply (lambda row: popularity(row), axis=1)
ses16m['popularity'] = ses16m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses16m['popularity'])
ses16m = pd.concat([ses16m, dummies], axis=1)
#ses16m.drop(['popularity'], inplace=True, axis=1)
ses16m.drop(['no'], inplace=True, axis=1)
ses16m.drop(['popular'], inplace=True, axis=1)
ses16m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
420,422,1.0,16,1,Florida,26.0,5,2.0,6.0,7.0,...,7.0,9.0,8.0,8.0,10.0,8.0,8.0,8.0,4.0,popular
421,423,2.0,16,1,Canada,30.0,2,2.0,1.0,1.0,...,5.0,4.0,6.0,7.0,9.0,8.0,9.0,9.0,3.0,no
422,424,3.0,16,1,TN,24.0,3,4.0,4.0,4.0,...,8.0,5.0,6.0,7.0,7.0,7.0,7.0,7.0,2.0,no
423,425,4.0,16,1,"New York, NY",28.0,0,2.0,3.0,7.0,...,2.0,2.0,6.0,6.0,9.0,6.0,9.0,9.0,2.0,no
424,426,5.0,16,1,New Jersey,24.0,1,2.0,4.0,7.0,...,4.0,2.0,7.0,8.0,8.0,8.0,10.0,9.0,2.0,no
425,427,6.0,16,1,California,28.0,1,2.0,1.0,1.0,...,1.0,10.0,6.0,7.0,10.0,8.0,7.0,9.0,0.0,no
426,428,7.0,16,1,New York,27.0,4,4.0,8.0,8.0,...,7.0,3.0,8.0,8.0,8.0,7.0,9.0,9.0,2.0,popular
427,429,8.0,16,1,"Pittsburgh, PA",27.0,3,2.0,1.0,1.0,...,7.0,6.0,7.0,8.0,8.0,9.0,9.0,8.0,3.0,no


In [84]:
#session 16 female (4)
ses16f = df2[(df2['wave'] == 16) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_16f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_16f_cutoff :
          return 'no'
ses16f.apply (lambda row: popularity(row), axis=1)
ses16f['popularity'] = ses16f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses16f['popularity'])
ses16f = pd.concat([ses16f, dummies], axis=1)
#ses16f.drop(['popularity'], inplace=True, axis=1)
ses16f.drop(['no'], inplace=True, axis=1)
ses16f.drop(['popular'], inplace=True, axis=1)
ses16f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
414,416,1.0,16,0,Miami,25.0,7,3.0,3.0,3.0,...,4.0,3.0,3.0,7.0,9.0,8.0,8.0,8.0,2.0,popular
415,417,2.0,16,0,Greece/Germany,29.0,1,2.0,1.0,1.0,...,3.0,8.0,5.0,7.0,8.0,4.0,8.0,8.0,3.0,no
416,418,3.0,16,0,New York,25.0,5,4.0,7.0,3.0,...,7.0,7.0,5.0,7.0,9.0,7.0,8.0,5.0,0.0,popular
417,419,4.0,16,0,Colorado,28.0,1,2.0,0.0,1.0,...,4.0,7.0,5.0,9.0,8.0,5.0,9.0,3.0,3.0,no
418,420,5.0,16,0,Indiana,22.0,7,2.0,1.0,1.0,...,6.0,4.0,5.0,8.0,9.0,7.0,10.0,7.0,2.0,popular
419,421,6.0,16,0,California,25.0,4,4.0,5.0,5.0,...,10.0,5.0,7.0,7.0,7.0,6.0,6.0,6.0,2.0,no


In [103]:
#session 17 male (5)
ses17m = df2[(df2['wave'] == 17) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_17m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_17m_cutoff :
          return 'no'
ses17m.apply (lambda row: popularity(row), axis=1)
ses17m['popularity'] = ses17m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses17m['popularity'])
ses17m = pd.concat([ses17m, dummies], axis=1)
#ses17m.drop(['popularity'], inplace=True, axis=1)
ses17m.drop(['no'], inplace=True, axis=1)
ses17m.drop(['popular'], inplace=True, axis=1)
ses17m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
438,440,1.0,17,1,"Milano, Italy",21.0,5,2.0,8.0,2.0,...,8.0,5.0,7.0,8.0,8.0,9.0,9.0,9.0,2.0,no
439,441,2.0,17,1,Boston,24.0,5,2.0,4.0,2.0,...,7.0,2.0,7.0,8.0,7.0,8.0,9.0,8.0,2.0,no
440,442,3.0,17,1,poland,25.0,4,2.0,1.0,1.0,...,2.0,1.0,6.0,7.0,9.0,8.0,8.0,9.0,3.0,no
441,443,4.0,17,1,New York,27.0,2,3.0,1.0,3.0,...,4.0,5.0,7.0,7.0,10.0,9.0,7.0,7.0,3.0,no
442,444,5.0,17,1,"Los Angeles, CA",26.0,5,2.0,5.0,1.0,...,5.0,6.0,3.0,8.0,7.0,9.0,8.0,7.0,0.0,no
443,445,6.0,17,1,"Burlington, Vermont",30.0,4,2.0,1.0,1.0,...,5.0,1.0,5.0,9.0,8.0,7.0,10.0,7.0,2.0,no
444,446,7.0,17,1,"Bogota, Colombia",28.0,5,3.0,2.0,2.0,...,3.0,3.0,7.0,6.0,8.0,9.0,7.0,5.0,3.0,no
445,447,8.0,17,1,India,26.0,3,4.0,1.0,1.0,...,2.0,4.0,3.0,6.0,10.0,6.0,7.0,4.0,1.0,no
446,448,9.0,17,1,New York City,22.0,6,2.0,1.0,1.0,...,6.0,5.0,3.0,8.0,9.0,8.0,9.0,10.0,4.0,popular
447,449,10.0,17,1,New York,23.0,3,6.0,7.0,1.0,...,4.0,1.0,5.0,7.0,10.0,10.0,10.0,8.0,3.0,no


In [86]:
#session 17 female (7)
ses17f = df2[(df2['wave'] == 17) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_17f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_17f_cutoff :
          return 'no'
ses17f.apply (lambda row: popularity(row), axis=1)
ses17f['popularity'] = ses17f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses17f['popularity'])
ses17f = pd.concat([ses17f, dummies], axis=1)
#ses17f.drop(['popularity'], inplace=True, axis=1)
ses17f.drop(['no'], inplace=True, axis=1)
ses17f.drop(['popular'], inplace=True, axis=1)
ses17f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
428,430,1.0,17,0,California,22.0,9,4.0,3.0,10.0,...,5.0,4.0,5.0,5.0,8.0,8.0,5.0,8.0,0.0,popular
429,431,2.0,17,0,Southern California,27.0,7,4.0,5.0,8.0,...,10.0,3.0,7.0,7.0,10.0,7.0,9.0,10.0,2.0,no
430,432,3.0,17,0,"Hastings-on-Hudson, NY",28.0,7,2.0,1.0,1.0,...,5.0,5.0,4.0,7.0,8.0,8.0,8.0,8.0,5.0,no
431,433,4.0,17,0,Northern California,30.0,6,6.0,3.0,3.0,...,5.0,3.0,5.0,8.0,6.0,7.0,8.0,5.0,3.0,no
432,434,5.0,17,0,Maryland,27.0,11,2.0,2.0,2.0,...,8.0,4.0,3.0,7.0,7.0,8.0,6.0,4.0,1.0,popular
433,435,6.0,17,0,New Jersey,22.0,4,1.0,1.0,1.0,...,10.0,10.0,5.0,7.0,9.0,9.0,9.0,9.0,4.0,no
434,436,7.0,17,0,California,23.0,7,2.0,9.0,1.0,...,10.0,4.0,5.0,9.0,8.0,10.0,7.0,6.0,4.0,no
435,437,8.0,17,0,North Carolina,23.0,4,2.0,2.0,6.0,...,5.0,6.0,5.0,7.0,9.0,9.0,9.0,8.0,5.0,no
436,438,9.0,17,0,California,23.0,10,2.0,1.0,5.0,...,8.0,3.0,4.0,7.0,8.0,7.0,7.0,7.0,10.0,popular
437,439,10.0,17,0,Baltimore,29.0,9,2.0,1.0,5.0,...,6.0,6.0,7.0,8.0,8.0,7.0,7.0,7.0,2.0,popular


In [87]:
#session 18 (3)
ses18 = df2[(df2['wave'] == 18)]

def popularity (row):
    if row['dec_o'] >= wave_18_cutoff :
          return 'popular'
    if row['dec_o'] < wave_18_cutoff :
          return 'no'
ses18.apply (lambda row: popularity(row), axis=1)
ses18['popularity'] = ses18.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses18['popularity'])
ses18 = pd.concat([ses18, dummies], axis=1)
#ses18.drop(['popularity'], inplace=True, axis=1)
ses18.drop(['no'], inplace=True, axis=1)
ses18.drop(['popular'], inplace=True, axis=1)
ses18

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
452,454,1.0,18,0,California,27.0,3,2.0,3.0,3.0,...,4.0,9.0,5.0,8.0,9.0,9.0,9.0,7.0,1.0,no
453,455,2.0,18,0,India,24.0,1,4.0,5.0,5.0,...,9.0,7.0,4.0,7.0,9.0,9.0,8.0,8.0,2.0,no
454,456,3.0,18,0,NJ,29.0,6,4.0,3.0,2.0,...,8.0,8.0,2.0,8.0,8.0,6.0,7.0,7.0,1.0,popular
455,457,4.0,18,0,"Santa Barbara, California",55.0,1,4.0,1.0,1.0,...,1.0,1.0,10.0,7.0,6.0,7.0,8.0,3.0,0.0,no
456,458,5.0,18,0,Hong Kong,29.0,2,4.0,9.0,8.0,...,10.0,9.0,5.0,10.0,10.0,9.0,10.0,10.0,5.0,no
457,459,6.0,18,0,New York City,26.0,4,4.0,6.0,1.0,...,8.0,1.0,5.0,5.0,5.0,5.0,5.0,8.0,0.0,popular
458,460,1.0,18,1,"Cincinnati, Ohio",33.0,5,2.0,2.0,1.0,...,1.0,5.0,8.0,8.0,8.0,7.0,9.0,7.0,1.0,popular
459,461,2.0,18,1,india,27.0,1,6.0,2.0,2.0,...,4.0,4.0,7.0,6.0,8.0,9.0,8.0,7.0,3.0,no
460,462,3.0,18,1,"Cambridge, MA",33.0,1,4.0,1.0,5.0,...,2.0,2.0,8.0,4.0,9.0,6.0,8.0,2.0,3.0,no
461,463,4.0,18,1,China,23.0,3,4.0,3.0,2.0,...,3.0,1.0,5.0,6.0,7.0,7.0,8.0,7.0,0.0,no


In [88]:
#session 19 male (8)
ses19m = df2[(df2['wave'] == 19) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_19m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_19m_cutoff :
          return 'no'
ses19m.apply (lambda row: popularity(row), axis=1)
ses19m['popularity'] = ses19m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses19m['popularity'])
ses19m = pd.concat([ses19m, dummies], axis=1)
#ses19m.drop(['popularity'], inplace=True, axis=1)
ses19m.drop(['no'], inplace=True, axis=1)
ses19m.drop(['popular'], inplace=True, axis=1)
ses19m

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
479,481,1.0,19,1,"Westchester County, N.Y.",23.0,2,2.0,8.0,1.0,...,6.0,1.0,6.0,8.0,7.0,8.0,9.0,5.0,4.0,no
480,482,2.0,19,1,Florida,30.0,9,2.0,2.0,6.0,...,4.0,3.0,4.0,7.0,10.0,9.0,8.0,8.0,3.0,no
481,483,3.0,19,1,Romania,28.0,5,2.0,1.0,1.0,...,10.0,5.0,5.0,8.0,10.0,10.0,10.0,10.0,3.0,no
482,484,4.0,19,1,Los Angeles,31.0,2,4.0,1.0,1.0,...,2.0,7.0,6.0,4.0,8.0,2.0,9.0,9.0,1.0,no
483,485,5.0,19,1,"South Orange, New Jersey",29.0,2,1.0,4.0,3.0,...,7.0,4.0,6.0,8.0,8.0,7.0,8.0,7.0,8.0,no
484,486,6.0,19,1,"Colombia, South America",32.0,6,3.0,1.0,1.0,...,2.0,2.0,8.0,7.0,10.0,8.0,9.0,9.0,1.0,no
485,487,7.0,19,1,New Jersey,24.0,1,4.0,1.0,1.0,...,6.0,2.0,5.0,7.0,9.0,6.0,8.0,5.0,2.0,no
486,488,8.0,19,1,India,29.0,4,4.0,1.0,1.0,...,8.0,3.0,5.0,5.0,10.0,7.0,10.0,8.0,4.0,no
487,489,9.0,19,1,Greece,28.0,11,2.0,6.0,5.0,...,6.0,5.0,6.0,6.0,8.0,8.0,9.0,10.0,2.0,popular
488,490,10.0,19,1,Northern New Jersey,29.0,2,2.0,6.0,1.0,...,7.0,4.0,3.0,3.0,7.0,6.0,8.0,6.0,3.0,no


In [89]:
#session 19 female (7)
ses19f = df2[(df2['wave'] == 19) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] > wave_19f_cutoff :
          return 'popular'
    if row['dec_o'] <= wave_19f_cutoff :
          return 'no'
ses19f.apply (lambda row: popularity(row), axis=1)
ses19f['popularity'] = ses19f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses19f['popularity'])
ses19f = pd.concat([ses19f, dummies], axis=1)
#ses19f.drop(['popularity'], inplace=True, axis=1)
ses19f.drop(['no'], inplace=True, axis=1)
ses19f.drop(['popular'], inplace=True, axis=1)
ses19f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
464,466,1.0,19,0,Texas/London,24.0,10,2.0,1.0,1.0,...,4.0,2.0,5.0,7.0,9.0,7.0,8.0,8.0,0.0,popular
465,467,2.0,19,0,"CT, FL, TN",24.0,10,2.0,7.0,4.0,...,9.0,9.0,7.0,8.0,8.0,8.0,8.0,8.0,4.0,popular
466,468,3.0,19,0,Detroit suburbs,25.0,10,6.0,5.0,2.0,...,7.0,6.0,4.0,8.0,9.0,6.0,8.0,8.0,2.0,popular
467,469,4.0,19,0,"Tokyo, Japan",25.0,11,4.0,1.0,1.0,...,6.0,9.0,4.0,8.0,8.0,8.0,6.0,6.0,6.0,popular
468,470,5.0,19,0,south carolina,29.0,11,4.0,1.0,1.0,...,10.0,1.0,4.0,4.0,9.0,5.0,8.0,6.0,4.0,popular
469,471,6.0,19,0,"California, New Jersey",30.0,7,4.0,8.0,8.0,...,10.0,4.0,5.0,7.0,8.0,8.0,9.0,9.0,2.0,no
470,472,7.0,19,0,California,25.0,7,3.0,1.0,3.0,...,5.0,9.0,4.0,8.0,10.0,10.0,8.0,9.0,12.0,no
471,473,8.0,19,0,Belgium,34.0,1,4.0,1.0,1.0,...,7.0,7.0,7.0,7.0,9.0,8.0,10.0,7.0,3.0,no
472,474,9.0,19,0,alabama,30.0,12,1.0,1.0,1.0,...,6.0,9.0,9.0,9.0,9.0,10.0,8.0,7.0,6.0,popular
473,475,10.0,19,0,San Francisco,27.0,8,3.0,1.0,2.0,...,6.0,9.0,5.0,5.0,8.0,5.0,7.0,7.0,2.0,no


In [104]:
#session 20 male (3)
ses20m = df2[(df2['wave'] == 20) & (df2['gender'] == 1)]

def popularity (row):
    if row['dec_o'] >= wave_20m_cutoff :
          return 'popular'
    if row['dec_o'] < wave_20m_cutoff :
          return 'no'
ses20m.apply (lambda row: popularity(row), axis=1)
ses20m['popularity'] = ses20m.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses20m['popularity'])
ses20m = pd.concat([ses20m, dummies], axis=1)
#ses20m.drop(['popularity'], inplace=True, axis=1)
ses20m.drop(['no'], inplace=True, axis=1)
ses20m.drop(['popular'], inplace=True, axis=1)
ses20m


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
500,502,1.0,20,1,Boston,26.0,0,4.0,1.0,1.0,...,3.0,4.0,3.0,7.0,7.0,9.0,9.0,9.0,6.0,no
501,503,2.0,20,1,California,25.0,0,2.0,1.0,5.0,...,7.0,7.0,5.0,7.0,9.0,4.0,10.0,5.0,1.5,no
502,504,4.0,20,1,Maryland,23.0,2,2.0,1.0,1.0,...,3.0,1.0,7.0,7.0,5.0,6.0,8.0,8.0,3.0,no
503,505,5.0,20,1,"Manila, Philippines",27.0,1,4.0,1.0,1.0,...,10.0,10.0,6.0,9.0,10.0,8.0,10.0,10.0,2.0,no
504,506,6.0,20,1,California,24.0,0,4.0,5.0,5.0,...,2.0,1.0,4.0,5.0,8.0,7.0,9.0,10.0,2.0,no
505,507,7.0,20,1,Florida,24.0,4,4.0,1.0,1.0,...,3.0,2.0,5.0,7.0,8.0,7.0,8.0,7.0,0.0,popular
506,508,8.0,20,1,New York,36.0,1,4.0,1.0,5.0,...,4.0,8.0,7.0,7.0,8.0,8.0,9.0,8.0,3.0,no


In [91]:
#session 20 female (4)
ses20f = df2[(df2['wave'] == 20) & (df2['gender'] == 0)]

def popularity (row):
    if row['dec_o'] >= wave_20f_cutoff :
          return 'popular'
    if row['dec_o'] < wave_20f_cutoff :
          return 'no'
ses20f.apply (lambda row: popularity(row), axis=1)
ses20f['popularity'] = ses20f.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses20f['popularity'])
#ses20f = pd.concat([ses20f, dummies], axis=1)
#ses20f.drop(['popularity'], inplace=True, axis=1)
#ses20f.drop(['popular'], inplace=True, axis=1)
#ses20f.drop(['no'], inplace=True, axis=1)
ses20f

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
494,496,1.0,20,0,Indiana,23.0,6,4.0,1.0,1.0,...,5.0,5.0,5.0,5.0,8.0,6.0,9.0,8.0,3.0,popular
495,497,2.0,20,0,"Boston, Ma",24.0,3,2.0,5.0,5.0,...,8.0,7.0,6.0,8.0,10.0,10.0,9.0,8.0,0.0,no
496,498,3.0,20,0,Baltimore,25.0,5,2.0,9.0,9.0,...,3.0,1.0,5.0,8.0,9.0,10.0,9.0,10.0,0.0,popular
497,499,4.0,20,0,New York,26.0,3,2.0,1.0,9.0,...,5.0,1.0,4.0,9.0,10.0,9.0,10.0,4.0,2.0,no
498,500,5.0,20,0,"Kansas City, Missouri",25.0,6,1.0,7.0,9.0,...,6.0,5.0,6.0,8.0,9.0,9.0,9.0,9.0,1.0,popular
499,501,6.0,20,0,Upstate New York,26.0,4,2.0,7.0,4.0,...,7.0,6.0,7.0,8.0,9.0,8.0,9.0,9.0,1.0,no


In [105]:
#session 21 (11)
ses21 = df2[(df2['wave'] == 21)]
cols = ["age"]
ses21[cols]=ses21[cols].fillna(ses21.mode().iloc[0])

def popularity (row):
    if row['dec_o'] >= wave_21_cutoff :
          return 'popular'
    if row['dec_o'] < wave_21_cutoff :
          return 'no'
ses21.apply (lambda row: popularity(row), axis=1)
ses21['popularity'] = ses21.apply (lambda row: popularity(row), axis=1)

dummies = pd.get_dummies(ses21['popularity'])
ses21 = pd.concat([ses21, dummies], axis=1)
#ses21.drop(['popularity'], inplace=True, axis=1)
ses21.drop(['no'], inplace=True, axis=1)
ses21.drop(['popular'], inplace=True, axis=1)
ses21

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
507,509,1.0,21,0,working,28.0,10,4.0,2.0,2.0,...,7.0,7.0,6.0,5.0,6.0,4.0,8.0,9.0,0.0,no
508,510,2.0,21,0,Chile,26.0,14,3.0,3.0,3.0,...,1.0,7.0,5.0,7.0,9.0,6.0,8.0,4.0,5.0,popular
509,511,3.0,21,0,India,23.0,4,4.0,2.0,2.0,...,7.0,8.0,6.0,8.0,4.0,10.0,8.0,8.0,10.0,no
510,512,4.0,21,0,"Cambridge, MA",27.0,9,2.0,1.0,1.0,...,5.0,5.0,5.0,7.0,8.0,8.0,7.0,7.0,2.0,no
511,513,5.0,21,0,Florida,23.0,15,2.0,1.0,1.0,...,6.0,6.0,2.0,6.0,8.0,7.0,6.0,4.0,2.0,popular
512,514,6.0,21,0,New York,23.0,4,2.0,8.0,9.0,...,6.0,7.0,7.0,8.0,10.0,8.0,9.0,7.0,5.0,no
513,515,7.0,21,0,Kansas,23.0,9,2.0,8.0,5.0,...,5.0,9.0,6.0,8.0,10.0,8.0,10.0,8.0,1.0,no
514,516,8.0,21,0,California,27.0,12,4.0,9.0,3.0,...,9.0,3.0,2.0,8.0,9.0,9.0,8.0,8.0,4.0,no
515,517,9.0,21,0,Mexico,30.0,6,3.0,1.0,1.0,...,3.0,9.0,3.0,8.0,9.0,7.0,9.0,7.0,2.0,no
516,518,10.0,21,0,San Diego,23.0,14,2.0,2.0,1.0,...,9.0,4.0,2.0,6.0,9.0,8.0,8.0,7.0,2.0,popular


In [109]:
cleaned_df_60 = ses1.append(ses2m).append(ses2f).append(ses3m).append(ses3f).append(ses3m).append(ses3f).append(ses4).append(ses5).append(ses6).append(ses7).append(ses8).append(ses9).append(ses10).append(ses11).append(ses12m).append(ses12f).append(ses13m).append(ses13f).append(ses14m).append(ses14f).append(ses15m).append(ses15f).append(ses16m).append(ses16f).append(ses17m).append(ses18).append(ses19m).append(ses19f).append(ses20m).append(ses20f).append(ses21)
cleaned_df_60


,iid,id,wave,gender,from,age,dec_o,race,imprace,imprelig,...,shopping,yoga,exphappy,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,match_es,popularity
0,1,1.0,1,0,Chicago,21.0,5,4.0,2.0,4.0,...,8.0,1.0,3.0,6.0,8.0,8.0,8.0,7.0,4.0,no
1,2,2.0,1,0,Alabama,24.0,6,2.0,2.0,5.0,...,3.0,1.0,4.0,7.0,5.0,10.0,8.0,3.0,3.0,popular
2,3,3.0,1,0,Connecticut,25.0,5,2.0,8.0,4.0,...,8.0,7.0,4.0,8.0,9.0,8.0,9.0,8.0,2.0,no
3,4,4.0,1,0,Texas,23.0,6,2.0,1.0,1.0,...,1.0,8.0,1.0,7.0,8.0,9.0,7.0,8.0,2.0,popular
4,5,5.0,1,0,Bowdoin College,21.0,3,2.0,8.0,1.0,...,8.0,3.0,7.0,6.0,3.0,6.0,10.0,8.0,2.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,548,18.0,21,1,Germany,30.0,10,2.0,8.0,8.0,...,4.0,1.0,7.0,10.0,10.0,10.0,10.0,10.0,5.0,no
547,549,19.0,21,1,Italy,28.0,10,2.0,2.0,3.0,...,8.0,2.0,5.0,7.0,10.0,8.0,9.0,9.0,3.0,no
548,550,20.0,21,1,UK,30.0,6,2.0,5.0,6.0,...,6.0,3.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,no
549,551,21.0,21,1,London & New York,27.0,10,1.0,2.0,1.0,...,7.0,1.0,3.0,7.0,8.0,3.0,8.0,5.0,3.0,no


In [111]:
#PICKLE
cleaned_df_60.to_pickle("./cleaned_df_60.pkl")